In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
print(os.listdir("../input"))


['data.npy', 'labels.npy']


In [2]:
import os
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [3]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
class MiniVGG:
    @staticmethod
    def build(width,height,depth,classes):
        model=Sequential()
        inputS=(height,width,depth)
        chanDim=-1
        if(K.image_data_format()=="channels_first"):
            inputS=(depth,height,width)
            chanDim=-1
        model.add(Conv2D(32,(3,3),padding="same",input_shape=inputS))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(32,(3,3),padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.25))
        model.add(Conv2D(64,(3,3),padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        model.add(Dense(classes))
        model.add(Activation("softmax"))


		return model

In [4]:
!pip install imutils
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
import tensorflow as tf
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
#from project.unet import Unet
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os
from keras import backend as K

tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)



EPOCHS = 16
INIT_LR = 1e-3 #Initial Learning rate
BS = 32 # Bach size to feed

# initialize the data and labels
print("Load images' NPY file")
data = []
labels = []
# grab the image paths and randomly shuffle them
random.seed(42)

tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)
data=np.load('../input/data.npy')
labels=np.load('../input/labels.npy')
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25, random_state=6)

# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,horizontal_flip=True, fill_mode="nearest")

# initialize the model
print("Compiling model...")
model = MiniVGG.build(width=128, height=128, depth=3, classes=2)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS) #Optimise uisng Adam 
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
    
# train the network
print("Training network")
#checkpointer = ModelCheckpoint('model-dsbowl2018-1.h5', verbose=1, save_best_only=True)
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX)//BS,
	epochs=EPOCHS, verbose=1,callbacks=[tensorboard])

label_name=["benign","malicious"]


# save the model to disk
print("Save the model for the applied CNN")
#model.save(args["model.hdf5"])


print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=128) #Check this
print(classification_report(testY.argmax(axis=1),
predictions.argmax(axis=1)))

cm = confusion_matrix(testY.argmax(axis=1), predictions.argmax(axis=1))
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS


  Stored in directory: /tmp/.cache/pip/wheels/b2/40/59/139d450e68847ef2f27d876d527b13389dac23df0f66526b5d
Successfully built imutils
Load images' NPY file
Compiling model...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Training network
Instructions for updating:
Use tf.cast instead.
Epoch 1/16
52/52 [==============================] - 8s 147ms/step - loss: 0.8508 - acc: 0.7644 - val_loss: 1.8057 - val_acc: 0.6523
Epoch 2/16
52/52 [==============================] - 6s 111ms/step - loss: 0.5429 - acc: 0.7800 - val_loss: 0.7664 - val_acc: 0.7240
Epoch 3/16
52/52 [==============================] - 6s 112ms/step - loss: 0.5111 - acc: 0.7881 - val_loss: 1.8651 - val_acc: 0.6039
Epoch 4/16
52/52 [==============================] - 6s 113ms/step - loss: 0.5054 - acc: 0.7926 - val_loss: 0.8842 - val_acc: 0.7832
Epoch 5/16
52/52 [============================

In [5]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
LOG_DIR = './logs' # Here you have to put your log directory
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2019-03-25 19:19:03--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.60.111, 52.21.103.149, 52.45.111.123, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.60.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13584026 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  12.95M  18.1MB/s    in 0.7s    

2019-03-25 19:19:04 (18.1 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13584026/13584026]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
http://0d7a9d6f.ngrok.io
